# PyTorch로 시작하는 딥러닝 입문
https://wikidocs.net/book/2788
## 01. 원-핫 인코딩
강아지 = [1, 0, 0]  
고양이 = [0, 1, 0]  
냉장고 = [0, 0, 1]  
원-핫 인코딩으로 표현된 벡터를 **원-핫 벡터(one-hot vector)** 라고함  
**원-핫 인코딩은 클래스를 0, 1, 2, 3, 4...로 분류했을 때 발생하는 가중치와 그에 따른 오차값을 해결하기 위해 사용**
## 02. 소프트맥스 회귀의 비용 함수 구현하기
### 1. 로우레벨

In [1]:
import torch
import torch.nn.functional as F
torch.manual_seed(1)

In [5]:
# 3x5 행렬 텐서
z = torch.rand(3, 5, requires_grad=True)

# 소프트맥스 함수 적용 단, 각 샘플에 대해서 적용해야하므로 두번째 차원에 대해서 적용한다는 의미의 dim=1 설정
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.1664, 0.1871, 0.1737, 0.2695, 0.2033],
        [0.2002, 0.1783, 0.2218, 0.1944, 0.2054],
        [0.1809, 0.2380, 0.2318, 0.1084, 0.2409]], grad_fn=<SoftmaxBackward>)


In [6]:
# 각 샘플에 대해서 임의의 레이블 생성
y = torch.randint(5, (3,)).long()
print(y)

tensor([3, 1, 2])


In [8]:
# unsqueeze(1)는 (3,) >> (3, 1)
print(y.unsqueeze(1))

tensor([[3],
        [1],
        [2]])


In [7]:
# 모든 원소가 0의 값을 가진 3 × 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis) 
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [9]:
print(y_one_hot)

tensor([[0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.]])


In [10]:
# 비용함수
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4992, grad_fn=<MeanBackward0>)


### 2. 하이레벨

- Low level : 소프트맥스 함수 결과에 로그를 씌우는 법  
    torch.log(F.softmax(z, dim=1))

In [11]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.7935, -1.6760, -1.7504, -1.3114, -1.5929],
        [-1.6086, -1.7244, -1.5062, -1.6381, -1.5826],
        [-1.7096, -1.4354, -1.4617, -2.2223, -1.4236]],
       grad_fn=<LogSoftmaxBackward>)

- Low level : 비용함수 구하는 법
    - 첫번째 수식
        (y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean()
    - 두번째 수식
        (y_one_hot * - F.log_softmax(z, dim=1)).sum(dim=1).mean()

In [12]:
# High level
# 세번째 수식
# nll_loss(Negative Log Likelihood)
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4992, grad_fn=<NllLossBackward>)

In [13]:
# 네번째 수식 (최종)
# corss_entropy는 F.log_softmax()와 F.nll_loss()를 포함
F.cross_entropy(z, y)

tensor(1.4992, grad_fn=<NllLossBackward>)

## 03. 소프트맥스 회귀 구현하기
### 1. 하이레벨

In [14]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# torch.manual_seed(1)

In [15]:
# 데이터 생성
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [16]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


In [17]:
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True) # 클래스 갯수 3
b = torch.zeros(1, requires_grad=True)

# optim 설정
optimizer = optim.SGD([W, b], lr=0.1)

In [19]:
n_epochs = 1000

for epoch in range(n_epochs+1):
    # cost
    z = x_train.matmul(W)+b
    cost = F.cross_entropy(z, y_train)
    
    # update
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%100==0:
        print(f'Epoch {epoch}/{n_epochs}\tCost {cost.item():.4f}')

Epoch 0/1000	Cost 1.0986
Epoch 100/1000	Cost 0.7611
Epoch 200/1000	Cost 0.6900
Epoch 300/1000	Cost 0.6432
Epoch 400/1000	Cost 0.6041
Epoch 500/1000	Cost 0.5683
Epoch 600/1000	Cost 0.5339
Epoch 700/1000	Cost 0.5003
Epoch 800/1000	Cost 0.4669
Epoch 900/1000	Cost 0.4335
Epoch 1000/1000	Cost 0.4000


### 2. 소프트맥스 회귀 nn.Module로 구현하기

In [20]:
model = nn.Linear(4, 3)
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [21]:
n_epochs = 1000

for epoch in range(n_epochs+1):
    
    # H
    prediction = model(x_train)
    
    # cost
    cost = F.cross_entropy(prediction, y_train)
    
    # update
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%100==0:
        print(f'Epoch {epoch}/{n_epochs}\tCost {cost.item():.4f}')

Epoch 0/1000	Cost 1.6168
Epoch 100/1000	Cost 0.6589
Epoch 200/1000	Cost 0.5734
Epoch 300/1000	Cost 0.5182
Epoch 400/1000	Cost 0.4733
Epoch 500/1000	Cost 0.4335
Epoch 600/1000	Cost 0.3966
Epoch 700/1000	Cost 0.3609
Epoch 800/1000	Cost 0.3254
Epoch 900/1000	Cost 0.2892
Epoch 1000/1000	Cost 0.2541


### 3. 소프트맥스 회귀 클래스로 구현하기

In [22]:
class SoftmaxClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)
        
    def forward(self, x):
        return self.linear(x)

In [23]:
model = SoftmaxClassifier()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [24]:
n_epochs = 1000

for epoch in range(n_epochs+1):
    
    # H
    prediction = model(x_train)
    
    # cost
    cost = F.cross_entropy(prediction, y_train)
    
    # update
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%100==0:
        print(f'Epoch {epoch}/{n_epochs}\tCost {cost.item():.4f}')

Epoch 0/1000	Cost 2.6376
Epoch 100/1000	Cost 0.6479
Epoch 200/1000	Cost 0.5646
Epoch 300/1000	Cost 0.5110
Epoch 400/1000	Cost 0.4672
Epoch 500/1000	Cost 0.4283
Epoch 600/1000	Cost 0.3919
Epoch 700/1000	Cost 0.3567
Epoch 800/1000	Cost 0.3216
Epoch 900/1000	Cost 0.2856
Epoch 1000/1000	Cost 0.2508


## 04. 소프트맥스 회귀로 MNIST 데이터 분류하기

In [25]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random

In [27]:
# 랜덤 시드 고정 for reproducibility
random.seed(777)
torch.manual_seed(777)

In [28]:
# hyperparameters
training_epochs = 15
batch_size = 100

In [30]:
# 데이터 세트 생성
mnist_train = dsets.MNIST(root='./MNIST_data/', # 데이터 다운로드 받을 경로
                          train=True, 
                          transform=transforms.ToTensor(), 
                          download=True)
mnist_test = dsets.MNIST(root='./MNIST_data/', 
                          train=False, 
                          transform=transforms.ToTensor(), 
                          download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST_data/MNIST/raw



/Users/limkahyun/anaconda3/envs/sona/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1631631181957/work/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [31]:
# data loader
data_loader = DataLoader(dataset=mnist_train,
                        batch_size=batch_size,
                         shuffle=True,
                         drop_last=True  # 마지막 배치를 버린다
                        )

In [34]:
# 모델 설계
# image shape = 28픽셀 * 28픽셀
# image lables = 10
model = nn.Linear(28*28, 10, bias=True)

In [35]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [38]:
# train
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = len(data_loader)
    
    for X, Y in data_loader:
        # 배치 크기가 100이므로 아래 연산에서 X는 (100, 784)의 텐서가 된다.
        X = X.view(-1, 28*28)
        
        # h
        h = model(X)
        
        # cost
        cost = criterion(h, Y)
        
        # update
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print(f'Epoch {epoch}/{training_epochs}\tCost {avg_cost:.4f}')

Epoch 0/15	Cost 0.5359
Epoch 1/15	Cost 0.3592
Epoch 2/15	Cost 0.3312
Epoch 3/15	Cost 0.3166
Epoch 4/15	Cost 0.3069
Epoch 5/15	Cost 0.3003
Epoch 6/15	Cost 0.2952
Epoch 7/15	Cost 0.2908
Epoch 8/15	Cost 0.2874
Epoch 9/15	Cost 0.2844
Epoch 10/15	Cost 0.2820
Epoch 11/15	Cost 0.2798
Epoch 12/15	Cost 0.2779
Epoch 13/15	Cost 0.2760
Epoch 14/15	Cost 0.2745


In [39]:
# test

# gradient 계산 수행 안함
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28*28).float()
    y_test = mnist_test.test_labels
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy: ', accuracy.item())
    
    # 무작위 하나 뽑아서 예측
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r+1].view(-1, 28*28).float()
    y_single_data = mnist_test.test_labels[r:r+1]
    print('Label: ', y_single_data.item())
    
    y_pred = model(X_single_data)
    print('y_pred: ', torch.argmax(y_pred, 1).item())

Accuracy:  0.8841999769210815
Label:  8
y_pred:  3


/Users/limkahyun/anaconda3/envs/sona/lib/python3.9/site-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/Users/limkahyun/anaconda3/envs/sona/lib/python3.9/site-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
